In [22]:
import psycopg2
import logging

logging.basicConfig(level=logging.INFO)

In [23]:
postgres_conn_ours = psycopg2.connect(
    host='127.0.0.1',
    user='postgres',
    password='16340954',
    database='bd_lv'
)

postgres_conn_m = psycopg2.connect(
    host='127.0.0.1',
    user='postgres',
    password='16340954',
    database='lv_m'
)

postgres_conn_a = psycopg2.connect(
    host='127.0.0.1',
    user='postgres',
    password='16340954',
    database='lv_a'
)

postgres_conn_f = psycopg2.connect(
    host='127.0.0.1',
    user='postgres',
    password='16340954',
    database='lv_f'
)

postgres_connections = [postgres_conn_ours, postgres_conn_m, postgres_conn_a, postgres_conn_f]

In [ ]:
extract_ours = ('''
  SELECT 
    veiculo.cd_carro,
    veiculo.nm_marca,
    veiculo.nm_motor,
    veiculo.ds_mecanizacao,
    veiculo.nm_modelo,
    veiculo.ds_ar_condicionado,
    veiculo.nm_cor,
    veiculo.nu_altura,
    veiculo.nu_tamanho,
    veiculo.nu_largura,
    veiculo.ds_foto,
    veiculo.nu_placa,
    veiculo.nu_chassi,
    veiculo.nm_combustivel,
    veiculo.nm_pneu,
    veiculo.nu_pressao_pneu,
    categoria.cd_categoria,
    categoria.nm_categoria,
    categoria.vl_valor_por_dia
  FROM 
    veiculo
  JOIN 
    categoria ON veiculo.cd_categoria = categoria.cd_categoria;
             
  SELECT *
  FROM cliente;
             
  SELECT *
  FROM locacao;
             
  SELECT *
  FROM patio;
             
  SELECT *
  FROM reserva;
''')

extract_m = ('''
  SELECT *
  FROM cliente;
             
  SELECT *
  FROM locacao;
             
  SELECT *
  FROM patio;
             
  SELECT *
  FROM reserva;
             
  SELECT 
      veiculo.id_veiculo,
      veiculo.placa,
      veiculo.marca,
      veiculo.modelo,
      veiculo.cor,
      veiculo.grupo_categoria,
      veiculo.ar_condicionado,
      veiculo.dimensoes,
      veiculo.foto_url,
      grupo_categoria.id_grupo,
      grupo_categoria.nome AS nome_grupo,
      grupo_categoria.faixa_valor_diaria
  FROM 
    veiculo
  JOIN 
    grupo_categoria ON veiculo.grupo_categoria = grupo_categoria.id_grupo;
''')

extract_a = ('''
  SELECT 
    clientes.id_cliente,
    clientes.nome,
    clientes.tipo_cliente,
    clientes.documento,
    endereco.rua,
    endereco.numero,
    endereco.bairro,
    endereco.cidade,
    endereco.estado,
    endereco.cep,
    endereco.complemento,
    clientes.telefone,
    clientes.email,
    clientes.data_nascimento,
    clientes.cnh,
    clientes.data_validade_cnh,
    clientes.cnpj_empresa
  FROM 
    clientes
  JOIN 
    endereco ON clientes.id_endereco = endereco.id_endereco;


  SELECT 
    veiculos.id_veiculo,
    veiculos.placa,
    veiculos.marca,
    veiculos.modelo,
    veiculos.cor,
    veiculos.ar_condicionado,
    veiculos.mecanizacao,
    veiculos.cadeirinha,
    veiculos.link_fotos,
    veiculos.assentos,
    veiculos.tamanho_mala,
    veiculos.carga_maxima,
    grupos_veiculos.id_grupo,
    grupos_veiculos.nome_grupo,
    prontuario.id_prontuario,
    prontuario.data_registro,
    prontuario.descricao_manutencao,
    prontuario.estado_conservacao,
    prontuario.pressao_pneus,
    prontuario.nivel_oleo
  FROM 
    veiculos
  JOIN 
    grupos_veiculos ON veiculos.id_grupo = grupos_veiculos.id_grupo;
  JOIN 
    prontuario ON veiculos.id_veiculo = prontuario.id_veiculo;
             
  SELECT *
  FROM locacoes;
             
  SELECT 
    endereco.id_endereco,
    endereco.rua,
    endereco.numero,
    endereco.bairro,
    endereco.cidade,
    endereco.estado,
    endereco.cep,
    endereco.complemento,
    patios.id_patio,
    patios.capacidade
  FROM 
    endereco
  JOIN 
    patios ON endereco.id_endereco = patios.id_endereco;
             
  SELECT *
  FROM reservas;
''')

extract_f = ('''
  SELECT 
    cliente.clienteid,
    cliente.nome AS clientenome,
    cliente.cpf_cnpj,
    cliente.endereco,
    cliente.telefone,
    cliente.email,
    cliente.tipo,
    condutor.condutorid,
    condutor.nome AS condutornome,
    condutor.cnh,
    condutor.dataexpiracaocnh
  FROM 
    cliente
  JOIN 
    condutor ON cliente.clienteid = condutor.clienteid;

  SELECT *
  FROM locacao;
             
  SELECT *
  FROM patio;
             
  SELECT *
  FROM reserva;
             
  SELECT *
  FROM veiculo;
''')

extract_queries = [extract_ours, extract_m, extract_a, extract_f]


In [29]:
def extract(postgres_connection, database_extract_query):
    try:
        cursor = postgres_connection.cursor()
        cursor.execute(database_extract_query)
        results = cursor.fetchall()
        print(results)
    except Exception as e:
        logging.error("An error occurred: %s", e)
        logging.debug("Failed query: %s", database_extract_query)
        postgres_connection.rollback()  # Roll back the transaction
    finally:
        cursor.close()  # Ensure the cursor is closed even if an error occurs

    return results

In [28]:
for connection, query in zip(postgres_connections, extract_queries):
   extract(connection, query)

[]
[]
[]
[]
